# Design Patterns with first class functions

## Strategy
OO version.  Concrete strategy is chosen by the client of the context class. System selects promotional strategy and passes it to the Order constructor.

In [12]:
from abc import ABCMeta, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelty')

class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantiy = quantity
        self.price = price
        
    def total(self):
        return self.price * self.quantity
    
class Order:  # the Context
    
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total, self.due)
    
class Promotion(ABCMeta):  # The Strategy: an abstract base class
    
    @abstractmethod
    def discount(self, order):
        """Return the discount as a postiive dollar amount"""
    
class FidelityPromo(Promotion): # first Concrete Strategy
    """ 5% discount for customers with 1000 or more Fidelity points"""
    
    def discount(self, order):
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0
    
class BulkItemPromo(Promotion):  # second Concrete Strategy
    """10% discount for each LineItem with 20 or more units"""

    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount


class LargeOrderPromo(Promotion):  # third Concrete Strategy
    """7% discount for orders with 10 or more distinct items"""

    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * .07
        return 0

In [15]:
joe = Customer('John Doe', 0)
joe

Customer(name='John Doe', fidelty=0)

In [13]:
ann = Customer('Ann Smith', 1100)

In [14]:
ann

Customer(name='Ann Smith', fidelty=1100)

Vs removing the abstract base class and passing in the actual funciton object

In [16]:

from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')


class LineItem:

    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity


class Order:  # the Context

    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)  # <1> Pass self back into the promo function, so we can access own attrs and fns
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

# <2>

def fidelity_promo(order):  # <3>
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0


def bulk_item_promo(order):
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount


def large_order_promo(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07


choose best strategy by iterating over a list of functions

In [18]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
    return max(promo(order) for promo in promos)  # can assign each promotion function a name, like n, as we iterate

finding strategies in a module

In [20]:
# use globals() to get a dictionary representing the global symbol table of current module
promos = [globals()[name] for name in globals()  # iterate over each dictionary returned by globals
              if name.endswith('_promo')   # select only those ending with '_promo' suffix
              and name != 'best_promo']     # filter out best promo itself, to avoid infinite recursion

def best_promo(order):
    return max(promo(order) for promo in promos)

In [21]:
promos

[<function __main__.fidelity_promo>,
 <function __main__.bulk_item_promo>,
 <function __main__.large_order_promo>]

Now, we can add more promos, and they'll be found automatically

Can also keep promos in their own module, and inspect it

In [ ]:
# import the promos module

promos = [func for name, func in 
             inspect.getmembers(promotions, inspect.isfunction)]

def best_promo(order):
    return max(promo(order) for promo in promos)

## Command

In [ ]:
class MacroCommand:
    """ A command that executes a list of commands """
    def __init__(self, commands):
        self.commands = list(commands)
    
    def __call__(self):
        for command in self.commands:
            command()